In [65]:
import numpy as np
import xgboost as xgb
import pandas as pd
from sklearn import datasets

In [66]:
iris=datasets.load_iris()
#Even number's data is for training
trainX=iris.data[1::2,:]
trainY=iris.target[1::2]
#Odd number's data is for test
testX=iris.data[2::2,:]
testY=iris.target[2::2]

In [67]:
pd.DataFrame(iris.data).head(4)

,0,1,2,3
0,5.1,3.5,1.4,0.2
1,4.9,3.0,1.4,0.2
2,4.7,3.2,1.3,0.2
3,4.6,3.1,1.5,0.2


In [68]:
pd.DataFrame(iris.target).head(4)

,0
0,0
1,0
2,0
3,0


In [69]:
#transform to DMatrix
trainX=pd.DataFrame(trainX)
trainX.columns=iris.feature_names
dtrain=xgb.DMatrix(trainX.as_matrix(),label=trainY.tolist())
testX=pd.DataFrame(testX)
testX.columns=iris.feature_names
dtest=xgb.DMatrix(testX.as_matrix())

In [70]:
trainX.head(4)

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm)
0,4.9,3.0,1.4,0.2
1,4.6,3.1,1.5,0.2
2,5.4,3.9,1.7,0.4
3,5.0,3.4,1.5,0.2


In [71]:
pd.DataFrame(trainY).head(4)

,0
0,0
1,0
2,0
3,0


In [72]:
dtrain.get_label()

array([ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,
        1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,
        1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  2.,  2.,
        2.,  2.,  2.,  2.,  2.,  2.,  2.,  2.,  2.,  2.,  2.,  2.,  2.,
        2.,  2.,  2.,  2.,  2.,  2.,  2.,  2.,  2.,  2.], dtype=float32)

In [73]:
np.random.seed(131)

In [74]:
params={'objective':'multi:softprob',
       'eval_metric':'mlogloss',
       'eta':0.3, # default=0.3
       'max_depth':6, #max depth of tree [default=6]
       'min_child_weight':1, #[default=1]
       'subsample':1,#[default=1]
       'num_class':3
       }

In [75]:
#do training
bst=xgb.train(params,dtrain,num_boost_round=18)

In [76]:
# calc variable score
imp=bst.get_fscore()
print(imp)
#The folloing results are represeted by 'f(number of column)'

{'f1': 4, 'f0': 3, 'f2': 70, 'f3': 39}


In [77]:
#Cross Validation, separate into 10 groups, boost iteration num = 40
cv=xgb.cv(params,dtrain,num_boost_round=40,nfold=10)
cv.head(4)

/home/vagrant/.pyenv/versions/anaconda3-4.0.0/lib/python3.5/site-packages/xgboost/training.py:272: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
  idset = [randidx[(i * kstep): min(len(randidx), (i + 1) * kstep)] for i in range(nfold)]


,test-mlogloss-mean,test-mlogloss-std,train-mlogloss-mean,train-mlogloss-std
0,0.771650,0.038101,0.753921,0.008132
1,0.568656,0.053669,0.545685,0.007138
2,0.436034,0.068086,0.408324,0.006829
3,0.346780,0.086041,0.313613,0.006850


In [78]:
#do training
#Return xgboost.core.Booster class
#In xgboost, two unique class is defined, one is DMatrix and the other is Booster
bst=xgb.train(params,dtrain,num_boost_round=18)
#do prediction
#predict is Booster's method
pred=bst.predict(dtest)
pred=pd.DataFrame(pred)

/home/vagrant/.pyenv/versions/anaconda3-4.0.0/lib/python3.5/site-packages/xgboost/core.py:840: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
  preds = preds.reshape(nrow, preds.size / nrow)


In [79]:
type(bst)

xgboost.core.Booster

In [80]:
pred.head(4)

,0,1,2
0,0.980634,0.011621,0.007745
1,0.980634,0.011621,0.007745
2,0.980634,0.011621,0.007745
3,0.980634,0.011621,0.007745


In [81]:
from sklearn.metrics import confusion_matrix
#make a confusion matrix for validation
pd.DataFrame(confusion_matrix(testY, pred.idxmax(axis=1)))

,0,1,2
0,24,0,0
1,0,21,4
2,0,1,24


In [82]:
#Comparison elements in the same column
#Return the maximum element's row index in each column
pred.idxmax(axis=1).head(6)

0    0
1    0
2    0
3    0
4    0
5    0
dtype: int64

In [83]:
pd.DataFrame(testY).head(6)

,0
0,0
1,0
2,0
3,0
4,0
5,0
